# TP1 - Aprendizaje Por Refuerzos
## Integrantes:
- **Azul Noguera** 
- **Paula Jordan** 
- **Rocio Gonzalez**
- **Valentina Pancaldi**

## Enunciado

Implementar un agente de aprendizaje por refuerzo y un ambiente en el cual el agente puede entrenarse. Esto incluye definir la noción de estado del ambiente, que puede no coincidir con el estado del juego.

## Preparación del Entorno

In [ ]:
# importamos las clases
from utils import *
from jugador import * # importamos la clase Jugador
from diezmil import * # importamos la clase DiezMil

# importamos funciones de utilidad
from utils import puntaje_y_no_usados, separar, JUGADA_PLANTARSE, JUGADA_TIRAR, JUGADAS_STR

# importamos las librerias necesarias
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import random
from random import randint
import csv

## Estado

In [ ]:
class EstadoDiezMil:
    def __init__(self):
        """Definir qué hace a un estado de diez mil.
        Recordar que la complejidad del estado repercute en la complejidad de la tabla del agente de q-learning.
        """
        self.dados = []  # Lista de los valores actuales de los dados
        self.puntaje_actual = 0  # Puntaje acumulado en el turno actual
        self.turno_terminado = False  # Flag para indicar si el turno ha terminado

    def reset(self):
        """Reinicia el estado del juego a sus valores iniciales."""
        self.dados = self.lanzar_dados(6)  # Reiniciar los dados
        self.puntaje_actual = 0  # Reiniciar el puntaje acumulado
        self.turno_terminado = False  # Reiniciar el flag de turno terminado

    def lanzar_dados(self, cantidad: int) -> list[int]:
        """Simula el lanzamiento de una cantidad específica de dados.
        Args:
            cantidad (int): La cantidad de dados a lanzar.
        Returns:
            list[int]: Los valores obtenidos en los dados lanzados.
        """
        return [randint(1, 6) for _ in range(cantidad)]

    def obtener_estado(self):
        """Devuelve el estado actual del juego en un formato que el agente pueda utilizar."""
        return (self.dados, self.puntaje_actual, self.turno_terminado)

    def actualizar_estado(self, nuevos_dados: list[int], puntaje: int) -> None:
        """Modifica las variables internas del estado luego de una tirada.
        Args:
            nuevos_dados (list[int]): Dados no usados tras la tirada.
            puntaje (int): Puntaje obtenido en la tirada.
        """
    
        self.dados = nuevos_dados
        self.puntaje_actual += puntaje
        if len(self.dados) == 0:
            # Si no quedan dados no usados, el turno debe terminar
            self.turno_terminado = True
    
    def fin_turno(self):
        """Modifica el estado al terminar el turno.
        """
        self.turno_terminado = True  # Marca el turno como terminado

    def __str__(self):
        """Representación en texto de EstadoDiezMil.
        Ayuda a tener una versión legible del objeto.

        Returns:
            str: Representación en texto de EstadoDiezMil.
        """
        estado_str = f"Dados: {self.dados}, Puntaje Actual: {self.puntaje_actual}, Turno Terminado: {self.turno_terminado}"
        return estado_str   

## Ambiente

In [ ]:
class AmbienteDiezMil:
    
    def __init__(self):
        """Definir las variables de instancia de un ambiente.
        ¿Qué es propio de un ambiente de 10.000?
        """
        self.estado = EstadoDiezMil()
        self.turno_terminado = False  # flag que indica si el turno terminó
        self.puntaje_total = 0  # Mantener un registro del puntaje total
        self.pasos_totales = 0  # Contador de pasos
        self.episodios_totales = 0  # Contador de episodios

    def reset(self):
        """
        Reinicia el estado del ambiente.
        """
        if self.pasos_totales > 0:  # Solo sumar episodios si se han realizado pasos
            self.episodios_totales += 1
        self.estado.reset()
        self.turno_terminado = False
        self.pasos_totales = 0
        
        # Aquí no reseteamos `self.puntaje_total` para mantener el acumulado entre episodios.
        return self.estado.obtener_estado()
    
    def step(self, accion):
        """
        Dada una acción devuelve una recompensa.
        El estado es modificado acorde a la acción y su interacción con el ambiente.
        Podría ser útil devolver si terminó o no el turno.
        
        Args:
            accion: Acción elegida por un agente.
        
        Returns:
            tuple[int, bool]: Una recompensa y un flag que indica si terminó el turno. 
        """
        # print(f"\n{'='*50}")
        # print(f"Acción Elegida: {accion.upper()}")

        # print(f"Estado Antes de la Acción: {self.estado}")
        
        self.pasos_totales += 1  # Incrementar el contador de pasos en cada acción
        # print(f"Paso {self.pasos_totales}: Acción = {accion}, Puntaje Total = {self.puntaje_total}")
        
        if accion == 'plantarse':
            self.turno_terminado = True
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            
        elif accion == 'tirar':
            nuevos_dados = self.estado.lanzar_dados(len(self.estado.dados))
            puntaje, dados_no_usados = puntaje_y_no_usados(nuevos_dados)
            # print(f"Tirada de Dados: {nuevos_dados}")
            # print(f"Puntaje Calculado: {puntaje}, Dados No Usados: {dados_no_usados}")
            self.estado.actualizar_estado(dados_no_usados, puntaje)
            recompensa = self.estado.puntaje_actual
            self.puntaje_total += recompensa  # Acumular la recompensa al puntaje total
            if recompensa == 0:
                self.turno_terminado = True
        else:
            raise ValueError("Acción no válida.")
        
        # print(f"Estado Después de la Acción: {self.estado}")
        # print(f"Recompensa Obtenida: {recompensa}")
        # print(f"Puntaje Total Acumulado: {self.puntaje_total}")
        # print(f"{'='*50}\n")
        
        return self.estado.obtener_estado(), recompensa, self.turno_terminado

## Agente

In [ ]:
class AgenteQLearning:
    def __init__(self, ambiente: AmbienteDiezMil, alpha: float, gamma: float, epsilon: float):
        self.ambiente = ambiente
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def elegir_accion(self, estado):
        # Convertir cada parte del estado a una tupla si es necesario
        estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
        
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['plantarse', 'tirar'])
        else:
            if estado in self.q_table:
                return max(self.q_table[estado], key=self.q_table[estado].get)
            else:
                return random.choice(['plantarse', 'tirar'])

    def entrenar(self, episodios: int, verbose: bool = False) -> None:
        for episodio in tqdm(range(episodios)):
            estado = self.ambiente.reset()
            estado = tuple(tuple(part) if isinstance(part, list) else part for part in estado)
            terminado = False

            while not terminado:
                accion = self.elegir_accion(estado)
                nuevo_estado, recompensa, terminado = self.ambiente.step(accion)
                nuevo_estado = tuple(tuple(part) if isinstance(part, list) else part for part in nuevo_estado)

                if estado not in self.q_table:
                    self.q_table[estado] = {'plantarse': 0, 'tirar': 0}
                
                if nuevo_estado not in self.q_table:
                    self.q_table[nuevo_estado] = {'plantarse': 0, 'tirar': 0}

                mejor_q_nuevo_estado = max(self.q_table[nuevo_estado].values())
                self.q_table[estado][accion] += self.alpha * (
                    recompensa + self.gamma * mejor_q_nuevo_estado - self.q_table[estado][accion]
                )

                estado = nuevo_estado

                if verbose:
                    print(f"Paso: {episodio + 1}, Estado: {estado}, Acción: {accion}, Recompensa: {recompensa}")

            # # Verificar la condición de finalización del juego
            if self.ambiente.puntaje_total >= 10000:
                print(f"Entrenamiento terminado en el episodio {episodio + 1} con un puntaje total de {self.ambiente.puntaje_total}.")
                break

    def guardar_politica(self, filename: str):
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            # Guardar encabezado
            writer.writerow(['estado', 'plantarse', 'tirar'])
            for estado, acciones in self.q_table.items():
                # Ordenar los dados dentro del estado antes de convertirlo en cadena
                dados_ordenados = tuple(sorted(estado[0]))  # Suponiendo que los dados son el primer elemento del estado
                estado_ordenado = (dados_ordenados,) + estado[1:]  # Recombina el estado con los dados ordenados
                estado_str = str(estado_ordenado)  # Convertir el estado a una cadena para almacenarlo en el CSV
                writer.writerow([estado_str, acciones['plantarse'], acciones['tirar']])

## Entrenamiento

In [ ]:
class JugadorEntrenado(Jugador):
    def __init__(self, nombre: str, filename_politica: str):
        self.nombre = nombre
        self.politica = self._leer_politica(filename_politica)
        
    def _leer_politica(self, filename:str, SEP:str=','):
        """Carga una politica entrenada con un agente de RL, que está guardada
        en el archivo filename en formato CSV.

        Args:
            filename (str): Nombre/Path del archivo que contiene a una política almacenada. 
        """
        politica = {}
        with open(filename, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                estado = eval(row['estado'])  
                politica[estado] = {
                    'plantarse': float(row['plantarse']),
                    'tirar': float(row['tirar'])
                }
        return politica
    
    def jugar(self, puntaje_total:int, puntaje_turno:int, dados:list[int]) -> tuple[int,list[int]]:
        """
        Devuelve una jugada y los dados a tirar.
        Args:
            puntaje_total (int): Puntaje total del jugador en la partida.
            puntaje_turno (int): Puntaje en el turno del jugador
            dados (list[int]): Tirada del turno.
        Returns:
            tuple[int,list[int]]: Una jugada y la lista de dados a tirar.
        """

        # Calcular el puntaje actual y los dados no usados
        puntaje, no_usados = puntaje_y_no_usados(dados)

        # Definir el estado como una tupla con los componentes relevantes
        estado = (tuple(dados), puntaje_turno, False)  # Matching the structure used in training
        
        # Consultar la política para determinar la jugada
        jugada = self.politica.get(estado, None) 
        if jugada is not None:
            accion = max(jugada, key=jugada.get)  # Choose the action with the highest value
        else:
            print(f"Estado {estado} no encontrado en la política, usando acción por defecto.")
            accion = 'plantarse'  # Default action if the state is not found

        if accion == 'plantarse':
            return ('plantarse', [])
        elif accion == 'tirar':
            return ('tirar', no_usados)

## Ejecución

In [ ]:
ambiente = AmbienteDiezMil()
agente = AgenteQLearning(ambiente, alpha=0.1, gamma=0.9, epsilon=0.2)
agente.entrenar(episodios=100, verbose=True)

## Guardado de Política

In [ ]:
agente.guardar_politica("politica_qlearning.csv")

## Evaluación

In [ ]:
jugador = JugadorEntrenado("Agente Trained", "politica_qlearning.csv")
print(jugador.politica)
resultado = jugador.jugar(puntaje_total=0, puntaje_turno=0, dados=[4, 2, 5, 2, 1, 5])

print(resultado)

## Calculo promedio de varias partidas

### 1

Hiperparametros:
- alpha=0.1
- gamma=0.9
- epsilon=0.2

In [ ]:
import numpy as np
from tqdm import tqdm

# Definir el número de partidas
numero_partidas = 1000  # Puedes ajustar este número según tus necesidades

# Almacenar los puntajes y el número de pasos de cada partida
puntajes_finales = []
episodios_por_partida = []

for _ in tqdm(range(numero_partidas)):
    # Crear una nueva instancia del ambiente y del agente para cada partida
    ambiente = AmbienteDiezMil()
    agente = AgenteQLearning(ambiente, alpha=0.1, gamma=0.9, epsilon=0.2)
    
    # Entrenar el agente en una partida de 100 episodios
    agente.entrenar(episodios=100, verbose=False)
    
    # Guardar el puntaje final de la partida y el número de pasos
    puntajes_finales.append(ambiente.puntaje_total)
    episodios_por_partida.append(ambiente.episodios_totales)  # Asumiendo que `ambiente.pasos_totales` guarda los pasos realizados

# Guardar la política después de todas las partidas
agente.guardar_politica("politica_qlearning.csv")

# Calcular el promedio de puntajes y el promedio de pasos
promedio_puntajes = np.mean(puntajes_finales)
promedio_pasos = np.mean(episodios_por_partida)

print(f"Promedio de puntajes después de {numero_partidas} partidas: {promedio_puntajes}")
print(f"Promedio de pasos por partida después de {numero_partidas} partidas: {promedio_pasos}")

In [ ]:
print(f"Promedio de puntajes después de {numero_partidas} partidas: {promedio_puntajes}")
print(f"Promedio de pasos por partida después de {numero_partidas} partidas: {promedio_pasos}")

In [ ]:
print(puntajes_finales)
print(episodios_por_partida)